In [28]:
import pandas as pd
import numpy as np
import os
import xarray as xr
import rasterio
from glob import glob
import geopandas as gpd

path_fire = r"C:\Users\anfel\OneDrive\Desktop\M2\prjt\data\algeria_tunisia.csv"
df_fire = pd.read_csv(path_fire)

df_fire_clean = df_fire[['longitude', 'latitude', 'type']].copy()

print(df_fire_clean.head())

   longitude  latitude  type
0    9.87200  27.72818     2
1    9.77909  28.63288     2
2    9.77953  28.64012     2
3    6.25409  36.74886     2
4    4.44782  35.87978     2


In [29]:
# Créer l'attribut binaire 'fire'
# Si 'type' == 0, 'fire' = 1, sinon 'fire' = 0
df_fire_clean['fire'] = np.where(df_fire_clean['type'] == 0, 1, 0)

print(f"df_fire_clean avec colonne 'fire' créé : {df_fire_clean.shape}")
print(df_fire_clean.head())

df_fire_clean avec colonne 'fire' créé : (90250, 4)
   longitude  latitude  type  fire
0    9.87200  27.72818     2     0
1    9.77909  28.63288     2     0
2    9.77953  28.64012     2     0
3    6.25409  36.74886     2     0
4    4.44782  35.87978     2     0


## Chargement et agrégation des données climatiques par saisons + Conversion en DataFrame

In [30]:
climate_csv_path = r"C:\Users\anfel\OneDrive\Desktop\M2\prjt\data\climat_clean.csv"
climate_df = pd.read_csv(climate_csv_path)

print("CSV climat chargé :", climate_df.shape)

# Convertir la date
climate_df["time"] = pd.to_datetime(climate_df["time"])

seasons_indices = {
    's1': [12, 1, 2],   # Hiver (Dec-Jan-Feb)
    's2': [3, 4, 5],    # Printemps
    's3': [6, 7, 8],    # Été
    's4': [9, 10, 11]   # Automne
}

# Ajouter colonne "season"

def get_season(date):
    m = date.month
    if m in [12, 1, 2]:
        return "s1"
    if m in [3, 4, 5]:
        return "s2"
    if m in [6, 7, 8]:
        return "s3"
    return "s4"

climate_df["season"] = climate_df["time"].apply(get_season)

all_dfs = []

variables = ["log_precip",  "tmax","amplitude_thermique"]

for var in variables:
    print(f"\nTraitement de la variable : {var}")

    for season in ["s1", "s2", "s3", "s4"]:
        
        df_season = climate_df[climate_df["season"] == season]

        if df_season.empty:
            print(f"   Saison {season} vide pour {var}")
            continue

        # Agrégation
        if var == "log_precip":
            df_agg = (
                df_season.groupby(["longitude", "latitude"])[var]
                .sum()
                .reset_index()
                .rename(columns={var: f"{var}_{season}"})
            )
        else:
            df_agg = (
                df_season.groupby(["longitude", "latitude"])[var]
                .mean()
                .reset_index()
                .rename(columns={var: f"{var}_{season}"})
            )

        all_dfs.append(df_agg)

        print(f"  → {var}_{season} agrégé ({df_agg.shape[0]} points)")


print("\n Toutes les variables saisonnières climatiques ont été générées.")

CSV climat chargé : (148292, 6)

Traitement de la variable : log_precip
  → log_precip_s1 agrégé (20301 points)
  → log_precip_s2 agrégé (20231 points)

Traitement de la variable : log_precip
  → log_precip_s1 agrégé (20301 points)
  → log_precip_s2 agrégé (20231 points)
  → log_precip_s3 agrégé (16200 points)
  → log_precip_s4 agrégé (21540 points)

Traitement de la variable : tmax
  → tmax_s1 agrégé (20301 points)
  → tmax_s2 agrégé (20231 points)
  → log_precip_s3 agrégé (16200 points)
  → log_precip_s4 agrégé (21540 points)

Traitement de la variable : tmax
  → tmax_s1 agrégé (20301 points)
  → tmax_s2 agrégé (20231 points)
  → tmax_s3 agrégé (16200 points)
  → tmax_s4 agrégé (21540 points)

Traitement de la variable : amplitude_thermique
  → amplitude_thermique_s1 agrégé (20301 points)
  → amplitude_thermique_s2 agrégé (20231 points)
  → amplitude_thermique_s3 agrégé (16200 points)
  → amplitude_thermique_s4 agrégé (21540 points)

 Toutes les variables saisonnières climatiques ont

## Fusion des DataFrames climatiques en un seul DataFrame

In [31]:
# Commencer avec le premier DataFrame
df_climat_merged = all_dfs[0]

# Fusionner tous les autres DataFrames sur longitude et latitude
for i, df in enumerate(all_dfs[1:], 1):
    df_climat_merged = pd.merge(df_climat_merged, df, on=['longitude', 'latitude'], how='inner')
    print(f"   → Fusion {i}/{len(all_dfs)-1} terminée")

print(f"\n DataFrame climatique fusionné créé : {df_climat_merged.shape}")
print(f" Colonnes : {list(df_climat_merged.columns)}")
print("\n Aperçu :")
print(df_climat_merged.head())

   → Fusion 1/11 terminée


   → Fusion 2/11 terminée
   → Fusion 3/11 terminée
   → Fusion 4/11 terminée
   → Fusion 5/11 terminée
   → Fusion 6/11 terminée
   → Fusion 7/11 terminée
   → Fusion 8/11 terminée
   → Fusion 9/11 terminée
   → Fusion 10/11 terminée
   → Fusion 11/11 terminée

 DataFrame climatique fusionné créé : (10113, 14)
 Colonnes : ['longitude', 'latitude', 'log_precip_s1', 'log_precip_s2', 'log_precip_s3', 'log_precip_s4', 'tmax_s1', 'tmax_s2', 'tmax_s3', 'tmax_s4', 'amplitude_thermique_s1', 'amplitude_thermique_s2', 'amplitude_thermique_s3', 'amplitude_thermique_s4']

 Aperçu :
   longitude   latitude  log_precip_s1  log_precip_s2  log_precip_s3  \
0     -8.625  27.291667       3.082025       0.916291       0.832909   
1     -8.625  27.375000       3.224341       0.916291       1.964311   
2     -8.625  27.458333       3.655782       1.321756       1.996060   
3     -8.625  27.541667       3.654403       1.267652       1.163151   
4     -8.625  27.625000       2.014903       1.357123       1.

## Merge final : Fire + Climat (approche optimisée avec pd.merge)

In [32]:
# Arrondir les coordonnées à la résolution de la grille climatique
# La grille climatique a un pas de 0.083333° (= 1/12°)
# On arrondit donc à 1 décimale après avoir divisé par 0.083333

resolution = 1/12  # 0.083333°

# Fonction d'arrondissement à la grille la plus proche
def round_to_grid(value, resolution):
    return np.round(value / resolution) * resolution

# Créer des copies avec coordonnées arrondies
df_fire_rounded = df_fire_clean.copy()
df_fire_rounded['longitude'] = round_to_grid(df_fire_clean['longitude'], resolution)
df_fire_rounded['latitude'] = round_to_grid(df_fire_clean['latitude'], resolution)

df_climat_rounded = df_climat_merged.copy()
df_climat_rounded['longitude'] = round_to_grid(df_climat_merged['longitude'], resolution)
df_climat_rounded['latitude'] = round_to_grid(df_climat_merged['latitude'], resolution)


print(f"   Avant : lon={df_fire_clean['longitude'].iloc[0]:.6f}, lat={df_fire_clean['latitude'].iloc[0]:.6f}")
print(f"   Après : lon={df_fire_rounded['longitude'].iloc[0]:.6f}, lat={df_fire_rounded['latitude'].iloc[0]:.6f}")

# Merge avec les coordonnées arrondies
df_final = pd.merge(
    df_fire_clean,  # Garder les coordonnées originales du feu
    df_climat_rounded,
    left_on=[df_fire_rounded['longitude'], df_fire_rounded['latitude']],
    right_on=['longitude', 'latitude'],
    how='inner',
    suffixes=('', '_y')
)

# Supprimer les colonnes dupliquées avec suffixe _y
cols_to_drop = [col for col in df_final.columns if col.endswith('_y')]
if cols_to_drop:
    df_final = df_final.drop(columns=cols_to_drop)
    print(f" Colonnes supprimées : {cols_to_drop}")

# Renommer key_0 et key_1 en longitude et latitude si elles existent
if 'key_0' in df_final.columns and 'key_1' in df_final.columns:
    df_final = df_final.rename(columns={'key_0': 'longitude', 'key_1': 'latitude'})

print(f"\n Dataset final : {df_final.shape}")
print(f" Colonnes : {list(df_final.columns)}")
print("\n Aperçu :")
print(df_final.head())

   Avant : lon=9.872000, lat=27.728180
   Après : lon=9.833333, lat=27.750000
 Colonnes supprimées : ['longitude_y', 'latitude_y']

 Dataset final : (21621, 16)
 Colonnes : ['longitude', 'latitude', 'type', 'fire', 'log_precip_s1', 'log_precip_s2', 'log_precip_s3', 'log_precip_s4', 'tmax_s1', 'tmax_s2', 'tmax_s3', 'tmax_s4', 'amplitude_thermique_s1', 'amplitude_thermique_s2', 'amplitude_thermique_s3', 'amplitude_thermique_s4']

 Aperçu :
   longitude  latitude  type  fire  log_precip_s1  log_precip_s2  \
0    6.25409  36.74886     2     0      15.060379      10.837062   
1    4.44782  35.87978     2     0      10.603816       8.966598   
2    5.53337  35.70751     0     1       9.746102       8.214118   
3    5.53337  35.70751     0     1      10.890906       9.116275   
4    3.17942  32.74046     2     0       3.105147       3.506458   

   log_precip_s3  log_precip_s4    tmax_s1    tmax_s2    tmax_s3    tmax_s4  \
0       4.965455      12.134838  16.666667  21.416667  30.916667  25.5

## Land Cover 

In [35]:
# Charger landcover
gdf_land = gpd.read_file(r"C:\Users\anfel\OneDrive\Desktop\M2\prjt\data\landcover_final_ML.gpkg")

# Calculer centroid du polygone
gdf_land["centroid"] = gdf_land.geometry.centroid

# Extraire lon/lat depuis le centroid
gdf_land["longitude"] = gdf_land["centroid"].x
gdf_land["latitude"]  = gdf_land["centroid"].y


C:\Users\anfel\AppData\Local\Temp\ipykernel_11596\1465160029.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_land["centroid"] = gdf_land.geometry.centroid


In [36]:
resolution = 1/12

def round_to_grid(value, resolution):
    return np.round(value / resolution) * resolution

gdf_land["lon_round"] = round_to_grid(gdf_land["longitude"], resolution)
gdf_land["lat_round"] = round_to_grid(gdf_land["latitude"],  resolution)


In [37]:
df_final["lon_round"] = round_to_grid(df_final["longitude"], resolution)
df_final["lat_round"] = round_to_grid(df_final["latitude"],  resolution)


In [38]:
df_full = df_final.merge(
    gdf_land[["lon_round", "lat_round", "GRIDCODE", "pays", "log_area_sqm", "lcc_code_encoded"]],
    on=["lon_round", "lat_round"],
    how="left"
)


In [39]:
df_full = df_full.drop(columns=["lon_round", "lat_round"], errors="ignore")


In [41]:
print("Shape final :", df_full.shape)
print(df_full.head())

Shape final : (442782, 20)
   longitude  latitude  type  fire  log_precip_s1  log_precip_s2  \
0    6.25409  36.74886     2     0      15.060379      10.837062   
1    6.25409  36.74886     2     0      15.060379      10.837062   
2    6.25409  36.74886     2     0      15.060379      10.837062   
3    6.25409  36.74886     2     0      15.060379      10.837062   
4    6.25409  36.74886     2     0      15.060379      10.837062   

   log_precip_s3  log_precip_s4    tmax_s1    tmax_s2    tmax_s3  tmax_s4  \
0       4.965455      12.134838  16.666667  21.416667  30.916667     25.5   
1       4.965455      12.134838  16.666667  21.416667  30.916667     25.5   
2       4.965455      12.134838  16.666667  21.416667  30.916667     25.5   
3       4.965455      12.134838  16.666667  21.416667  30.916667     25.5   
4       4.965455      12.134838  16.666667  21.416667  30.916667     25.5   

   amplitude_thermique_s1  amplitude_thermique_s2  amplitude_thermique_s3  \
0                8.41666